In [1]:
from aug.features import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_questions_by_lesion(lesion: str):
    questions = [
        # f"What is {lesion}?", # don't need the first one for extending features.
        f"What are the symptoms associated with {lesion}?",
        f"What can cause {lesion}?",        
        f"What are the patient’s symptoms that are relevant for {lesion}?",
        f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
        f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
        f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
        f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
    ]
    return questions

In [3]:
lesion = "atelectasis"
## adding prior knowledge from 8 questions.
questions = get_questions_by_lesion(lesion)
documents = request_documents(lesion)
responses = neo4jvector_get_responses(questions, documents) # let's try to get the responses from other LLMs to predict the keywords.
keywords = responses_to_keywords(lesion, responses)
keywords

Node properties are the following:

Relationship properties are the following:

The relationships are the following:



{'age',
 'asthma',
 'asymptomatic',
 'blockage of a bronchiole or bronchus',
 'breathing difficulty',
 'bronchoscopy',
 'certain types of surgery',
 'chest pain',
 'chest x-ray',
 'chronic obstructive pulmonary disease',
 'compression from the outside',
 'cough',
 'ct chest',
 'cyanosis',
 'fever',
 'foreign body',
 'high oxygen',
 'increased heart rate',
 'low oxygen saturation',
 'lower lung segments',
 'mucus plug',
 'muscle relaxation',
 'obesity',
 'old age',
 'pleural effusion',
 'poor surfactant spreading',
 'post-surgical complications',
 'proximal obstruction',
 'pulmonary tuberculosis',
 'small volume linear shadows',
 'smoking',
 'surfactant deficiency',
 'tumor',
 'type of anesthetic'}

In [4]:
responses

{'What are the symptoms associated with atelectasis?': 'The symptoms associated with atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, atelectasis can also be asymptomatic. Fever is not a symptom of atelectasis.',
 'What can cause atelectasis?': 'Atelectasis can be caused by various medical conditions, including post-surgical complications, surfactant deficiency, and poor surfactant spreading during inspiration. It can also be caused by blockage of a bronchiole or bronchus, such as by a foreign body, mucus plug, tumor, or compression from the outside. Risk factors for atelectasis include certain types of surgery, muscle relaxation, obesity, high oxygen, lower lung segments, age, chronic obstructive pulmonary disease or asthma, and type of anesthetic.',
 'What are the patient’s symptoms that are relevant for at

In [5]:
# identified_keywords = {
#     "boolean": [
#         "blockage of the airways",
#         "chest pain",
#         "chronic obstructive pulmonary disease (copd)",
#         "collapse or closure of a lung",
#         "coughing",
#         "foreign object",
#         "pneumonia",
#         "prolonged bed rest",
#         "rapid breathing",
#         "reduced or absent gas exchange",
#         "shortness of breath",
#     ],
#     "numerical": ["oxygen saturation (%)"],  # 'low oxygen levels',
#     # 'lung diseases',
#     # 'mucus',
#     # 'pneumonia',
#     # 'surgery',
#     # 'tumor'
# }

In [6]:
# identified_keywords = {  # networkx
#     "boolean": [
#         "abnormal chest x-ray findings",
#         "abnormal lung sounds",
#         "collapsed lung tissue" "bluish skin color",
#         "chest pain",
#         "coughing",
#         "coughing up blood",
#         "crackling sounds in the lungs",
#         "decreased appetite",
#         "decreased breath sounds",
#         "decreased oxygen saturation",
#         "difficulty breathing",
#         "difficulty speaking",
#         "dyspnea",
#         "fatigue",
#         "history of being on mechanical ventilation",
#         "history of chest trauma or injury",
#         "history of chronic obstructive pulmonary disease (copd)",
#         "history of lung cancer",
#         "history of lung diseases",
#         "history of neuromuscular disorders",
#         "history of prolonged bed rest or immobility",
#         "history of recent surgery",
#         "history of respiratory infections",
#         "history of smoking for 20 years",
#         "advanced age",
#         "low oxygen levels",
#         "obesity",
#         "fungi in sputum culture",
#         "presence of bacteria",
#         "rapid breathing",
#         "shortness of breath",
#         "smoking",
#         "tachypnea",
#         "underlying lung diseases",
#         "viruses",
#         "wheezing",
#         "white-out appearance in the affected area of the lung",
#     ],
#     "numerical": ["levels of oxygen and carbon dioxide in the blood"],
#     #  'amount of air that can be inhaled and exhaled',
#     #  'use of accessory muscles',
#     #  'viruses',
# }

In [7]:
identified_keywords = {  # Neo4jVector
    "boolean": [
        "anesthesia",
        "asthma",
        "asymptomatic",
        "breathing difficulty",
        "chest pain",
        "chronic obstructive pulmonary disease",
        "cough",
        "cyanosis",
        "fever",
        "pleural effusion",
        "pulmonary tuberculosis",
        "small volume linear shadows",
        "smoking"
        # Response: Risk factors for atelectasis include certain types of surgery, muscle relaxation, obesity, high oxygen, and lower lung segments.',
        # "high oxygen",  # A cause, not a symptom. Has the patient been the situation with high oxygen. # remove this for not causing confusion to LLMs.

    ],

    "numerical": [
        "heart rate (per minute)",  # "increased heart rate",
        "oxygen saturation (%)",  # "low oxygen saturation",
    ],
    # 'old age', # already provided in the report.
}

In [15]:
import pandas as pd
df = pd.read_csv('./spreadsheets/reflacx_clinical.csv')
sample_df = df.iloc[:50]

In [16]:
results = []
all_res = []
all_prompt = []
for idx, data in sample_df.iterrows():
    prompt, res, result = get_possible_values(
        data,
        identified_keywords,
        responses=responses,
    )
    all_prompt.append(prompt)
    all_res.append(res)
    results.append(result)

In [17]:
for f in identified_keywords['numerical'] + identified_keywords['boolean']:
    sample_df[f] = None
    sample_df[f] = [ r[f] for r in results]

In [18]:
sample_df.to_csv('[prior-knowledge] neo4j-vector.csv')

In [21]:
sample_df[sample_df['Atelectasis'] >0]['oxygen saturation (%)'].mean(), sample_df[sample_df['Atelectasis'] <=0]['oxygen saturation (%)'].mean()

(93.05882352941177, 96.36363636363636)

In [22]:
sample_df[sample_df['Atelectasis'] >0]['chest pain'].mean(), sample_df[sample_df['Atelectasis'] <=0]['chest pain'].mean()

(0.23529411764705882, 0.18181818181818182)

In [29]:
sample_df[sample_df['Atelectasis'] >0]['fever'].mean(), sample_df[sample_df['Atelectasis'] <=0]['fever'].mean()


(0.0, 0.030303030303030304)

In [13]:
print(all_prompt[0]['boolean'])

**Prior Knowledge:**
Question: What are the symptoms associated with atelectasis?
Answer: The symptoms associated with atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, atelectasis can also be asymptomatic. Fever is not a symptom of atelectasis.
Question: What can cause atelectasis?
Answer: Atelectasis can be caused by various medical conditions, including post-surgical complications, surfactant deficiency, and poor surfactant spreading during inspiration. It can also be caused by blockage of a bronchiole or bronchus, such as by a foreign body, mucus plug, tumor, or compression from the outside. Risk factors for atelectasis include certain types of surgery, muscle relaxation, obesity, high oxygen, lower lung segments, age, chronic obstructive pulmonary disease or asthma, and type of anesthetic.
Question: What 

In [14]:
print(all_prompt[0]['numerical'])

**Prior Knowledge:**
Question: What are the symptoms associated with atelectasis?
Answer: The symptoms associated with atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, atelectasis can also be asymptomatic. Fever is not a symptom of atelectasis.
Question: What can cause atelectasis?
Answer: Atelectasis can be caused by various medical conditions, including post-surgical complications, surfactant deficiency, and poor surfactant spreading during inspiration. It can also be caused by blockage of a bronchiole or bronchus, such as by a foreign body, mucus plug, tumor, or compression from the outside. Risk factors for atelectasis include certain types of surgery, muscle relaxation, obesity, high oxygen, lower lung segments, age, chronic obstructive pulmonary disease or asthma, and type of anesthetic.
Question: What 